<h1 align='center'>Process Individual Fire Workspace<h1>

#### ANALYST ACTION: paste IR camera data to "/_camera_data/" folder

In [ ]:
### Block 1 ### Optional: open ABWF web dashboard

import webbrowser
AB_wildfire_link = 'https://www.arcgis.com/apps/dashboards/3ffcc2d0ef3e4e0999b0cf8b636defa3'
webbrowser.open(AB_wildfire_link)
print("COMPLETED - browser window opened")

In [ ]:
### Block 2 ### Input outside variables

fireName = input("fire name, if no name exists leave empty")

import re
def scan_time_validator():
    while True:
        time = input("24HR time. Format: '0200'")
        pattern = r"[0-2][0-9][0-9][0-9]"
        if re.match(pattern, time):
            return time
        else:
            assert False,"ERROR: invalid time format, please try input again"

# Example usage
captureTime = scan_time_validator()

#captureTime = input("24HR time. Format: '0200'")
if fireName == "":
    fireName = "None entered"

print(f"COMPLETED - Fire name: {fireName} | Capture time: {captureTime}")

#### ANALYST ACTION: input fire (if exists) and fire scan start time

In [ ]:
### Block 3 ### Assign file names and directory paths

productionDirectory = r"C:\Verimap-IRmapper\.Alberta_mapping\production_workspace"
gdrive = r"G:\Shared drives\GIS Data Center\Delivered To The Goverment\Alberta_deliverables"

import datetime, os
#Generate datetimes
delivery_date_raw = datetime.date.today()
delivery_date = delivery_date_raw.strftime('%y%m%d')
if int(captureTime) > 1600:
    scan_date_raw = delivery_date_raw - datetime.timedelta(days = 1)
else:
    scan_date_raw = delivery_date_raw

#Capture name of this fire from file name
script_name = os.path.basename(globals()['__vsc_ipynb_file__'])
fireID = script_name[:6]
if fireName == None:
    fireName = fireID 

## folder paths
masterFireDir = f'{productionDirectory}/{delivery_date}/{fireID}_{delivery_date}'
deliverablesDir = f'{masterFireDir}/{fireID}'
rawSensorDataDir = f'{productionDirectory}/{delivery_date}/_camera_data_{delivery_date}/{fireID}'
recentFirePerimeterFilepath = f'{masterFireDir}/Scratch/{fireID}_request_perimeter.shp'

## BUILD FILE NAMES
breadcrumbFileName = f'{fireID}{delivery_date[2:]}{captureTime[:2]}b'
hotspotsFileName = f'{fireID}{delivery_date[2:]}{captureTime[:2]}h'
perimeterFileName = f'{fireID}{delivery_date[2:]}{captureTime[:2]}p'
imageryFileName = f'{fireID}{delivery_date[2:]}{captureTime[:2]}i'

print(f"COMPLETED - all file names and folder paths assigned")

In [ ]:
### Block 4 ### Push initial data to directories

import shutil
## Build deliverables directory
if fireID in os.listdir(masterFireDir):
   shutil.rmtree(deliverablesDir)
os.mkdir(deliverablesDir)

# Build data directories
for dir in ['Breadcrumb','Hotspots','Perimeter','QuicklookMap','Thermal_Imagery']:
   if dir not in os.listdir(deliverablesDir):
      os.mkdir(f"{deliverablesDir}/{dir}")

# Move files into appropriate locations
for file in os.scandir(f'{rawSensorDataDir}'):
   if file.name.endswith('.csv'):
      shutil.copy(file.path, f'{masterFireDir}/Scratch/{hotspotsFileName}.csv')
   if file.name.endswith('.kml'):
      shutil.copy(file.path, f'{masterFireDir}/Scratch/{breadcrumbFileName}.kml')
   if file.name.endswith('.tif'):
      shutil.copy(file.path, f'{deliverablesDir}/Thermal_Imagery/{imageryFileName}.tif')

open(f"{deliverablesDir}/QuicklookMap/{fireID}{delivery_date}HotspotDetection.pdf", "w+")

print("COMPLETED - Files are copied successfully")

In [ ]:
### Block 5 ### Clean and produce breadcrumb files

import geopandas, simplekml, fiona
# Read the beadcrumb KML file into a GeoDataFrame
kml_file = f"{masterFireDir}/Scratch/{breadcrumbFileName}.kml"
fiona.drvsupport.supported_drivers['KML'] = 'rw'
#geopandas.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gdf_breadcrumb = geopandas.read_file(kml_file, driver='KML')
gdf_breadcrumb.drop(1, axis=0, inplace=True)
gdf_breadcrumb.at[0,'Description'] = f"{fireID}"

# Write the GeoDataFrame to a shapefile
gdf_breadcrumb_3400 = gdf_breadcrumb.to_crs(3400)
gdf_breadcrumb_3400.to_file(f'{deliverablesDir}/Breadcrumb/{breadcrumbFileName}.shp', driver='ESRI Shapefile')

# Extract geometry data
gdf_breadcrumb['points'] = gdf_breadcrumb.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
lineString = gdf_breadcrumb.iloc[0]['points']
lineStringList = gdf_breadcrumb.iloc[0]['points']

# Create an instance of Kml
kml = simplekml.Kml(open=1)

# Create a linestring with two points (ie. a line)
lin = kml.newlinestring(name="Flight Path")
lin.coords = lineStringList
lin.style.linestyle.color = 'ff800080'
lin.style.linestyle.width = 2

kml.save(f'{deliverablesDir}/Breadcrumb/{breadcrumbFileName}.kml')

print("COMPLETED - Breadcrumb files were generated successfully")

In [ ]:
### Block 6 ### Clean and produce initial hotspot shapefile

import pandas
hotSpotTextFile = f'{masterFireDir}/Scratch/{hotspotsFileName}.csv'
df_hotSpots = pandas.read_csv(hotSpotTextFile, sep=",", header=None)
df_hotSpots.columns = df_hotSpots.iloc[0]

# Parse hotspots CSV table
if df_hotSpots.columns[0] == "Label":
    print("standard CSV")
    #Ingest standard hotspot CSV
    df_hotSpots = df_hotSpots.drop(0).drop(columns=["Label", "DDM_lat", "DDM_long"], axis=1).dropna(axis=1)
    df_hotSpots.columns = ["HotspotID","Latitude","Longitude","HeatScore"]
elif df_hotSpots.columns[0] == "Index":
    print("highres CSV")
    #Ingesting hotspotting CSV table
    df_hotSpots = df_hotSpots.drop(0).drop(columns=["Threshold","filename","col","row"], axis=1).dropna(axis=1)
    df_hotSpots.columns = ["HotspotID","HeatScore","Class","Longitude","Latitude"]
else:
    print("Hotspots CSV is a non-standard format")

df_hotSpots = df_hotSpots.astype({"HotspotID": "int64", 
                                  "Latitude": "float32", 
                                  "Longitude": "float32", 
                                  "HeatScore": "float32"})
x = scan_date_raw.strftime('%d-%b-%y')

templateColumns = {
    'Heat': 'HOTSPOT',
    'Fire_Num': f'{fireID}',	
    'Fire_Name': f'{fireName}',	
    'CaptureDat': f'{x}',	
    'CaptureTim': f'{captureTime[:2]}:{captureTime[2:]}:00',	
    'Agency': 'Verimap Plus Inc',
    'Comments': f'{fireName}',	
    'Contact': 'Amir Paz',	
    'Phone': '403-464-3059',	
    'Email': 'apaz@verimap.com'	
}

for pair in templateColumns:
    df_hotSpots[pair] = templateColumns[pair]

gdf_hotSpots = geopandas.GeoDataFrame(
    df_hotSpots, geometry= geopandas.points_from_xy(df_hotSpots.Longitude, 
                                             df_hotSpots.Latitude), 
                                             crs="EPSG:4269")
gdf_hotSpots = gdf_hotSpots.to_crs(3400)

# Function: convert from decimal degrees to degrees, minutes, seconds.
def deg_to_dms(row, sphere=''):
    if sphere == 'Latitude':
        deg = row['Latitude']
    if sphere == 'Longitude':
        deg = row['Longitude']
    m, s = divmod(abs(deg)*3600, 60)
    d, m = divmod(m, 60)
    if deg < 0:
        d = -d
    d, m = int(d), int(m)
    s = round(s,2)
    return f"{d}° {m}' {s}\""

# Apply the user-defined function to every row
gdf_hotSpots['Lat_DMS'] = gdf_hotSpots.apply(lambda row: deg_to_dms(row, sphere='Latitude'), axis=1)
gdf_hotSpots['Long_DMS'] = gdf_hotSpots.apply(lambda row: deg_to_dms(row, sphere='Longitude'), axis=1)

#export geodataframe as shapefile
gdf_hotSpots.to_file(f'{deliverablesDir}/Hotspots/{hotspotsFileName}.shp')
hotspots_count1 = len(gdf_hotSpots)

print("COMPLETED - Hotspot files were generated successfully")
#gdf_hotSpots.head()

In [ ]:
### Block 7 ### Open validation and mapping environment

os.system("start " + f'{productionDirectory}/{delivery_date}/{fireID}_{delivery_date}/{fireID}_{delivery_date}.qgz')
print("COMPLETED - mapping environment opened")

#### ANALYST ACTION: validate hotspots
<p>0. Open layers thermal img, recent_perimeter, hotspots, breadcrumb (in that order) to map</p>
<p>1. Remove outliers from the hotspots file</p>
<p>2. (if hotspotting) Turn HotspotID column into consecutive numbers. In field calculator, "update existing field" - "@row_number" in expression box</p>
<p>3. Save feature edits</p>
<p>4. Leave QGIS workspace open</p>

In [ ]:
### Block 8 ### Create final hotspot files

#Validation: check if number of hotspots changed
gdf_hotSpots = geopandas.read_file(f'{deliverablesDir}/Hotspots/{hotspotsFileName}.shp')
hotspots_count2 = len(gdf_hotSpots)
if hotspots_count1 == hotspots_count2:
    def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
    prRed('Hotspot count is the same as before editing.')
    prRed("Did you forget to save edits to the hotspots in QGIS? If all hotspots were valid, proceed.")

#Export hotSpots CSV deliverable
df_hotSpots = pandas.DataFrame(gdf_hotSpots.drop(columns='geometry'))
df_hotSpots.to_csv(f'{deliverablesDir}/Hotspots/{hotspotsFileName}.csv')

## Build data table and colorize KML file
kml = simplekml.Kml(open=1)

for index, row in df_hotSpots.iterrows():
    pnt = kml.newpoint(name=row['HotspotID'])
    pnt.extendeddata.newdata(name='HotspotID', value=row['HotspotID'])
    pnt.extendeddata.newdata(name='HeatScore', value=row['HeatScore'])
    pnt.extendeddata.newdata(name='Fire_Num', value=row['Fire_Num'])
    pnt.extendeddata.newdata(name='Fire_Name', value=row['Fire_Name'])
    pnt.extendeddata.newdata(name='CaptureDat', value=row['CaptureDat'])
    pnt.extendeddata.newdata(name='CaptureTim', value=row['CaptureTim'])
    pnt.extendeddata.newdata(name='Agency', value=row['Agency'])
    pnt.extendeddata.newdata(name='Comments', value=row['Comments'])
    pnt.extendeddata.newdata(name='Contact', value=row['Contact'])
    pnt.extendeddata.newdata(name='Phone', value=row['Phone'])
    pnt.extendeddata.newdata(name='Email', value=row['Email'])
    pnt.extendeddata.newdata(name='Lat_DMS', value=row['Lat_DMS'])
    pnt.extendeddata.newdata(name='Long_DMS', value=row['Long_DMS'])
    pnt.coords = [(row['Longitude'], row['Latitude'])]
    pnt.style.labelstyle.scale = 0
    pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/shaded_dot.png'
    pnt.style.iconstyle.scale = 0.5
    if row['HeatScore'] >= 55.0:
        pnt.style.iconstyle.color = simplekml.Color.red
    elif row['HeatScore'] >= 26.0:
        pnt.style.iconstyle.color = simplekml.Color.orange #'ff0000ff'
    elif row['HeatScore'] >= 12.0:
        pnt.style.iconstyle.color = simplekml.Color.yellow #'ff00a5ff'
    else:
        pnt.style.iconstyle.color = simplekml.Color.yellow #'ff00ffff'

kml.save(f'{deliverablesDir}/Hotspots/{hotspotsFileName}.kml')

print("COMPLETED - valid hotspots KML, SHP and CSV files were generated successfully")

#### ANALYST ACTION: If no perimeter update is needed, jump to map prodution section (starting at Block 13)

In [ ]:
### Block 9 ### Import hotspot geometries

## Make point geometry object from hotspots
import shapely
multiPointList = []
for x in range(len(gdf_hotSpots)):
    li = (shapely.get_x(gdf_hotSpots.geometry[x]), shapely.get_y(gdf_hotSpots.geometry[x]))
    multiPointList.append(li)
print("COMPLETED - analyst inspect object")
shapely.MultiPoint(multiPointList)


#### ANALYST ACTION: Calibrate alpha and buffer values for concave hull geometry build

In [ ]:
### Block 10 ### Calibrate values

alpha = float(input("enter Alpha parameter (if empty, default = 0.02)").strip() or 0.02)
print(f"Alpha parameter: {alpha}")

buffer_m = int(input("enter buffer parameter (if empty, default = 20m)").strip() or 20)
print(f"Buffer parameter: {buffer_m}meters")

geometry_perimeter_hull = shapely.concave_hull(shapely.MultiPoint(multiPointList), ratio=alpha)
geometry_perimeter_hull = shapely.buffer(geometry_perimeter_hull, buffer_m)
gdf_new_perimeter = geopandas.GeoDataFrame(index=[0], crs=gdf_hotSpots.crs, geometry=[geometry_perimeter_hull])
gdf_new_perimeter.to_file(f"{masterFireDir}/Scratch/{fireID}_burning_zone.shp")

geometry_perimeter_hull

#### ANALYST ACTION: in QGIS manually correct burning area

In [ ]:
### Block 11 ### Rebuild geometry from cleaned burning zone shapefile

# Make shape from recent perimeter shapefile
if os.path.exists(f"{recentFirePerimeterFilepath}"):
    with fiona.open(f"{recentFirePerimeterFilepath}") as shapefile:
        # Iterate over the records
        for record in shapefile:
            # Get the geometry from the record
            geometry_request_perim = shapely.geometry.shape(record['geometry'])
else:
    print("No request perimeter was submitted with this fire")

# Make shape from burning zone shapefile
with fiona.open(f"{masterFireDir}/Scratch/{fireID}_burning_zone.shp") as shapefile:
    # Iterate over the records
    for record in shapefile:
        # Get the geometry from the record
        geometry_burning_zone = shapely.geometry.shape(record['geometry'])
print("Inspect final burning zone geometry")
geometry_burning_zone

In [ ]:
### Block 12 ### Union perimeters and export final shapefiles

## Union recent perimeter and burning_zone
if os.path.exists(f"{recentFirePerimeterFilepath}"):
    if shapely.get_type_id(geometry_request_perim) == 6 or shapely.get_type_id(geometry_request_perim) == 3:
        geometry_perimeter = shapely.ops.unary_union([geometry_request_perim, geometry_burning_zone])
    else:
        geometry_perimeter = geometry_burning_zone
else:
    geometry_perimeter = geometry_burning_zone

## Attribute Perimeter
gdf_new_perimeter = geopandas.GeoDataFrame(index=[0], crs=gdf_hotSpots.crs, geometry=[geometry_perimeter])
perimeter_area = gdf_new_perimeter['geometry'].area/ 10**4
perimeter_length = gdf_new_perimeter['geometry'].length

# Add constant columns
templateColumns = {
    'FIRENUMBER': f'{fireID}-2023',
    'FIRE_NUMBE': f'{fireID}',
    'FIRE_CLASS': 'E',
    'BURNCODE': '',
    'BURN_CLASS': '',
    'HECTARES_U': int(perimeter_area.iloc[0]),
    'PERIM_m': int(perimeter_length.iloc[0]),
    'YEAR_DATE': '2023',
    'ALIAS': fireName,
    'CAPTURE_DA': scan_date_raw.strftime('%d-%b-%y'),
    'CLOCK': captureTime,
    'SOURCE': 6,
    'SOURCE_AGENCY': 'Verimap Plus Inc',
}

for pair in templateColumns:
    gdf_new_perimeter[pair] = templateColumns[pair]
gdf_new_perimeter.head()

## Export SHP & KML
gdf_new_perimeter.to_file(f"{deliverablesDir}/Perimeter/{perimeterFileName}.shp")
if os.path.exists(f'{deliverablesDir}/Perimeter/{perimeterFileName}.kml'):
    os.remove(f'{deliverablesDir}/Perimeter/{perimeterFileName}.kml')
gdf_new_perimeter.to_file(f'{deliverablesDir}/Perimeter/{perimeterFileName}.kml', driver='KML')

os.system(f"open -a 'qgis' {deliverablesDir}/Perimeter/{perimeterFileName}.shp")
print("COMPLETED - final shapefiles produced")
geometry_perimeter

## MAP PRODUCTION

#### ANALYST ACTION: Go make final maps in QGIS

In [ ]:
### Block 13 ### Run to copy text to clipboard for map layout info box

import pyperclip
x = scan_date_raw.strftime('%b %d %Y').upper()
fire_details_string = f"Fire Scan Code:\n{fireID}{delivery_date}\n\nAlias:\n{fireName}\n\nDate of Acquisition:\n{x}\n\nTime of Acquisition:\n{captureTime[:2]}:{captureTime[2:]}"
pyperclip.copy(fire_details_string)

In [ ]:
### Block 14 ### Zip and export client package

## folder path
month_numeric = datetime.date.today().strftime('%m')
month_literal = datetime.date.today().strftime('%B')
day_numeric = datetime.date.today().strftime('%d')
gdrive_deliverables = f'{gdrive}/{month_numeric}_{month_literal}/{month_literal.upper()}_{day_numeric}/'

# Remove if already loaded to gdrive
if os.path.exists(f'{gdrive_deliverables}{fireID}.zip'):
    os.remove((f'{gdrive_deliverables}{fireID}.zip'))

# remove .xml file
for root, dirs, files in os.walk(f'{deliverablesDir}'):
    for name in files:
        if name.endswith('.xml'):
            os.remove(os.path.join(root, name))
  
# Remove perimeter folder if empty
if os.path.exists(f'{deliverablesDir}/Perimeter'):
    if len(os.listdir(f'{deliverablesDir}/Perimeter')) == 0: 
        shutil.rmtree(f'{deliverablesDir}/Perimeter')
# Remove hotspot folder if empty
if os.path.exists(f'{deliverablesDir}/Hotspots'):
    if len(os.listdir(f'{deliverablesDir}/Hotspots')) == 0: 
        shutil.rmtree(f'{deliverablesDir}/Hotspots')

shutil.make_archive(deliverablesDir, 'zip', deliverablesDir)
shutil.move(f"{deliverablesDir}.zip", f'{gdrive_deliverables}')